In [1]:
from langchain_ollama import OllamaLLM

llm = OllamaLLM(model="mistral")


In [4]:
llm.invoke("what model are you")

" I am a model of the Mistral AI's family of models, specifically trained as a conversational agent."